#### Imports

In [1]:
%matplotlib inline
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import xarray as xr
import yaml

import gvpy as gv

%reload_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'

# Generate files for NASA archive

dataID_locationID_YYYYMMDD[hh[mm[ss]]]_R#[_L#][_V#][_comments].ict

Where the only allowed characters are: a-zA-Z0-9_.- (that is, upper case and lower case alphanumeric, underscore, period, and hyphen). All fields not in square brackets are required. Fields are described as follows:
```
    dataID: short description of measured parameter/species, instrument, or model (e.g., O3; RH; VOC; PTRMS; MM5)
    locationID: short description of site, station, platform, laboratory or institute
    YYYY: four-digit year
    MM: two-digit month
    DD: two-digit day
    hh: optional two-digit hour
    mm: optional two-digit minute
    ss: optional two-digit second
    R: revision number of data
    L: optional launch number
    V: optional volume number
    comments: optional additional information
    extension: ict file extension, always “ict”
```

## Copy data

Copy data to new directory. Use `cp -RL` to follow the symlinks in the git-annex data structure.

In [2]:
data_dir = Path("/Users/gunnar/Desktop/piston_upload/")

In [3]:
data_dir.mkdir(exist_ok=True)

In [21]:
!cp -RL ../data/proc/thermistor $data_dir/

In [40]:
!cp -RL ../data/proc/adcp $data_dir/

## Rename thermistor files

file name example:

PISTON-moored-temperature_mooring-SIO1_start-time_R0_grid_thru-end-time.nc

In [22]:
tdir = data_dir.joinpath("thermistor/L2")

In [23]:
files = sorted(tdir.glob("*.nc"))

In [24]:
files

[PosixPath('/Users/gunnar/Desktop/piston_upload/thermistor/L2/PISTON_SIO1_Tgrid.nc'),
 PosixPath('/Users/gunnar/Desktop/piston_upload/thermistor/L2/PISTON_SIO3_Tgrid.nc')]

In [25]:
for file in files:
    a = xr.open_dataarray(file)
    time_start = "".join(a.time_coverage_start.split(":")[0].split("T")[0].split("-"))
    time_end = "".join(a.time_coverage_end.split(":")[0].split("T")[0].split("-"))
    file_name = f"PISTON-moored-temperature_mooring-{a.mooring}_{time_start}_R0_grid_thru{time_end}.nc"
    new = file.parent.joinpath(file_name)
    file.rename(new)

## Rename ADCP files

In [41]:
adir = data_dir.joinpath("adcp")
moorings = sorted(adir.glob("SIO*"))

In [42]:
for mooring in moorings:
    files = sorted(mooring.glob("*.nc"))
    for file in files:
        a = xr.open_dataset(file)
        time_start = "".join(a.time_coverage_start.split(":")[0].split("T")[0].split("-"))
        time_end = "".join(a.time_coverage_end.split(":")[0].split("T")[0].split("-"))
        file_name = f"PISTON-moored-velocity-SN{a.sn}_mooring-{a.mooring}_{time_start}_R0_grid_thru{time_end}.nc"
        new = file.parent.joinpath(file_name)
        file.rename(new)